# rpy2

In [1]:
from rpy2 import robjects as ro
from rpy2.robjects import pandas2ri
from rpy2.robjects.packages import importr
import pandas as pd

**Install packages**

In [3]:
import rpy2.robjects.packages as rpackages
#utils = rpackages.importr('utils')
#utils.chooseCRANmirror(ind=1)
#utils.install_packages('fixest')

**From `pd.DataFrame` to `rpy2.DataFrame`**

In [4]:
pd_df = pd.DataFrame({'x1':[1,2,3], 'x2':['a','b','c']})

with (ro.default_converter + pandas2ri.converter).context():
  r_df = ro.conversion.get_conversion().py2rpy(pd_df)

r_df

x1,x2
...,...


**From `rpy2.DataFrame` dataframe to `pd.DataFrame`**

In [5]:
with (ro.default_converter + pandas2ri.converter).context():
  pd_df = ro.conversion.get_conversion().rpy2py(r_df)
pd_df

,x1,x2
0,1,a
1,2,b
2,3,c


**`fixest` estimation**

In [6]:
fixest = importr('fixest')
stats = importr('stats')
base = importr('base')

In [7]:
iris = pd.read_csv("https://raw.githubusercontent.com/mwaskom/seaborn-data/master/iris.csv")

with (ro.default_converter + pandas2ri.converter).context():
  iris_r = ro.conversion.get_conversion().py2rpy(iris)
iris_r

sepal_length,sepal_width,petal_length,petal_width,species
...,...,...,...,...


In [8]:
formula = ro.Formula('sepal_length~sepal_width+petal_length+petal_width|species')
result = fixest.feols(formula, data=iris_r)
print(result)

OLS estimation, Dep. Var.: sepal_length
Observations: 150 
Fixed-effects: species: 3
Standard-errors: Clustered (species) 
              Estimate Std. Error  t value Pr(>|t|)    
sepal_width   0.495889   0.120618  4.11124 0.054382 .  
petal_length  0.829244   0.097011  8.54796 0.013411 *  
petal_width  -0.315155   0.109610 -2.87524 0.102670    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
RMSE: 0.300627     Adj. R2: 0.862705
                 Within R2: 0.652007



In [36]:
result.names

'nobs','nobs_ori...,'fml',...,'ll_null','ssr_fe_o...,'ll_fe_on...


In [39]:
from rpy2.robjects.vectors import ListVector
my_list = ListVector({'key1': 'value1', 'key2': 'value2'})


In [49]:
d = pd.DataFrame({'x':['a','2',pd.NA]})
d

,x
0,a
1,2
2,<NA>


In [52]:
d.x.isna().mean()

0.3333333333333333

In [9]:
stats.coef(result)

0.495889,0.829244,-0.315155


In [10]:
stats.confint(result)

2.5 %,97.5 %
...,...


In [12]:
result.names

'nobs','nobs_ori...,'fml',...,'ll_null','ssr_fe_o...,'ll_fe_on...


In [110]:
base.summary(result)

<rpy2.robjects.vectors.ListVector object at 0x7f107e56a5c0> [RTYPES.VECSXP]
R classes: ('fixest',)
[IntSexpVe..., IntSexpVe..., LangSexpV..., LangSexpV..., ..., FloatSexp..., FloatSexp..., BoolSexpV..., ListSexpV...]
  nobs: <class 'rpy2.robjects.vectors.IntVector'>
  <rpy2.robjects.vectors.IntVector object at 0x7f109185dc40> [RTYPES.INTSXP]
R classes: ('integer',)
[150]
  nobs_origin: <class 'rpy2.robjects.vectors.IntVector'>
  <rpy2.robjects.vectors.IntVector object at 0x7f109185dfc0> [RTYPES.INTSXP]
R classes: ('integer',)
[150]
  fml: <class 'rpy2.robjects.Formula'>
  <rpy2.robjects.Formula object at 0x7f109185ddc0> [RTYPES.LANGSXP]
R classes: ('formula',)
  call: <class 'rpy2.robjects.language.LangVector'>
  Rlang( (function (fml, data, vcov, weights, offset, subset, split, fsplit,  )
...
  method: <class 'rpy2.robjects.vectors.FloatVector'>
  <rpy2.robjects.vectors.FloatVector object at 0x7f109185ddc0> [RTYPES.REALSXP]
R classes: ('numeric',)
[-111.725978]
  method_type: <class 'rpy2.robjects.vectors.FloatMatrix'>
  <rpy2.robjects.vectors.FloatMatrix object at 0x7f10915b5bc0> [RTYPES.REALSXP]
R classes: ('matrix', 'array')
[0.014549, -0.010154, 0.003056, -0.010154, ..., 0.003008, 0.003056, 0.003008, 0.012014]
  fml_all: <class 'rpy2.robjects.vectors.BoolVector'>
  <rpy2.robjects.vectors.BoolVector object at 0x7f107e5692c0> [RTYPES.LGLSXP]
R classes: ('logical',)
[       1]
<rpy2.robjects.vectors.ListVector object at 0x7f107e56a5c0> [RTYPES.VECSXP]
R classes: ('fixest',)
[IntSexpVe..., IntSexpVe..., LangSexpV..., LangSexpV..., ..., FloatSexp..., FloatSexp..., BoolSexpV..., ListSexpV...]

AttributeError: module 'base' has no attribute 'coeftable'

In [114]:
from rpy2.robjects.numpy2ri import pandas2ri
iris_r = pandas2ri.py2ri(iris)

ImportError: cannot import name 'pandas2ri' from 'rpy2.robjects.numpy2ri' (/bbkinghome/mbarrera/miniconda3/lib/python3.9/site-packages/rpy2/robjects/numpy2ri.py)

In [118]:
ro.globalenv["iris_r"] = iris_r

In [125]:
ro.r(
'''
library(fixest)
feols(sepal_length~petal_length+petal_width, iris_r)
'''
)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

<rpy2.robjects.vectors.ListVector object at 0x7f0fd2af5780> [RTYPES.VECSXP]
R classes: ('fixest',)
[IntSexpVe..., IntSexpVe..., LangSexpV..., LangSexpV..., ..., FloatSexp..., FloatSexp..., FloatSexp..., FloatSexp...]
  nobs: <class 'numpy.ndarray'>
  array([150], dtype=int32)
  nobs_origin: <class 'numpy.ndarray'>
  array([150], dtype=int32)
  fml: <class 'rpy2.robjects.Formula'>
  <rpy2.robjects.Formula object at 0x7f0fd2aad200> [RTYPES.LANGSXP]
R classes: ('formula',)
  call: <class 'rpy2.robjects.language.LangVector'>
  Rlang( feols(fml = sepal_length ~ petal_length + petal_width, data = iris_r) )
...
  method: <class 'numpy.ndarray'>
  array([0.7662613])
  method_type: <class 'numpy.ndarray'>
  array([102.16833333])
  fml_all: <class 'numpy.ndarray'>
  array([23.88069367])
  nparams: <class 'numpy.ndarray'>
  array([-184.03976641])

In [116]:
import pandas as pd
from rpy2.robjects import pandas2ri
from rpy2.robjects.packages import importr

# Activate the automatic conversion of pandas data frames to R data frames
pandas2ri.activate()

# Create a pandas data frame with the iris data
iris_df = pd.read_csv("https://raw.githubusercontent.com/mwaskom/seaborn-data/master/iris.csv")

# Import R's fixest package
fixest = importr("fixest")

# Run the R code with the iris_df as an argument
result = fixest.feols("sepal_length~sepal_width+petal_length+petal_width|species", iris_df)

# Print the result
print(result)

R[write to console]: Error in (function (fml, data, vcov, weights, offset, subset, split, fsplit,  : 
  The argument 'fml' must be a two-sided formula. Problem: it is not a
formula, not a data.frame nor a matrix (instead it is a vector).

R[write to console]: In addition: 
R[write to console]: Warning messages:

R[write to console]: 1: 
R[write to console]: In (function (package, help, pos = 2, lib.loc = NULL, character.only = FALSE,  :
R[write to console]: 
 
R[write to console]:  library ‘/usr/share/R/library’ contains no packages

R[write to console]: 2: 
R[write to console]: In (function (package, help, pos = 2, lib.loc = NULL, character.only = FALSE,  :
R[write to console]: 
 
R[write to console]:  library ‘/usr/share/R/library’ contains no packages

R[write to console]: 3: 
R[write to console]: In (function (package, help, pos = 2, lib.loc = NULL, character.only = FALSE,  :
R[write to console]: 
 
R[write to console]:  library ‘/usr/share/R/library’ contains no packages

R[write 

RRuntimeError: Error in (function (fml, data, vcov, weights, offset, subset, split, fsplit,  : 
  The argument 'fml' must be a two-sided formula. Problem: it is not a
formula, not a data.frame nor a matrix (instead it is a vector).
